In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import sklearn.metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.impute import KNNImputer

import pandas_profiling as pp
import scipy

In [2]:
#Import the data
df1 = pd.read_csv('archive/2014_Financial_Data.csv')
df2 = pd.read_csv('archive/2015_Financial_Data.csv')
df3 = pd.read_csv('archive/2016_Financial_Data.csv')
df4 = pd.read_csv('archive/2017_Financial_Data.csv')
df5 = pd.read_csv('archive/2018_Financial_Data.csv')

# Explore the data

At this point it would make sense to explore the data, see if there are any missing values and get a better sense of the features/dimensions and everything.

In [3]:
df1.shape

(3808, 225)

In [4]:
df2.shape

(4120, 225)

In [5]:
df3.shape

(4797, 225)

In [6]:
df4.shape

(4960, 225)

In [7]:
df5.shape

(4392, 225)

So over 5 years we have 3808 + 4120 + 4797 + 4960 + 4392 = **22,707 observations**. We have 225 features, let's get a good sense of what they are.

In [8]:
df1.head()

,Unnamed: 0,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,...,Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,2015 PRICE VAR [%],Class
0,PG,7.440100e+10,-0.0713,3.903000e+10,3.537100e+10,0.000000e+00,2.146100e+10,2.146100e+10,1.391000e+10,7.090000e+08,...,-0.0187,-0.0217,0.0359,0.0316,0.1228,0.0000,-0.1746,Consumer Defensive,-9.323276,0
1,VIPS,3.734148e+09,1.1737,2.805625e+09,9.285226e+08,1.083303e+08,3.441414e+08,7.939267e+08,1.345959e+08,1.214869e+07,...,NaN,NaN,NaN,NaN,NaN,1.6484,1.7313,Consumer Defensive,-25.512193,0
2,KR,9.837500e+10,0.0182,7.813800e+10,2.023700e+10,0.000000e+00,1.519600e+10,1.751200e+10,2.725000e+09,4.430000e+08,...,0.0618,0.0981,0.1886,0.3268,0.2738,0.0000,0.0234,Consumer Defensive,33.118297,1
3,RAD,2.552641e+10,0.0053,1.820268e+10,7.323734e+09,0.000000e+00,6.561162e+09,6.586482e+09,7.372520e+08,4.245910e+08,...,0.0211,-0.0510,-0.0189,0.1963,-0.0458,0.0000,-0.0060,Consumer Defensive,2.752291,1
4,GIS,1.790960e+10,0.0076,1.153980e+10,6.369800e+09,0.000000e+00,3.474300e+09,3.412400e+09,2.957400e+09,3.024000e+08,...,0.0257,0.0090,0.0215,0.0274,0.1025,0.0000,-0.0220,Consumer Defensive,12.897715,1


So many features! We have Revenue, Revenue Growth, Cost of Revenue, Operating expenses etc. There are many good tactics here to avoid overfitting but before we move on we should probably look at these features in more detail (try to see if there's anything we're missing). We also appear to have an unnamed column (which is the column of stocks) and missing values! 

Ok, now let's decide some strategy. What should our train and test sets be?

Intuition says we have 5 dataframes with a total of 22K observations. Proposed Strategy for this document is:

1. 2014 - 2016: **Train Set** (3 Years)
2. 2017 - 2018: **Test Set** (2 years)

Side Note: Could even use the API to scrape 2019 data to see how we do!

## What type of problem is this?

This is a classification problem. [Kaggle](https://www.kaggle.com/cnic92/200-financial-indicators-of-us-stocks-20142018) has an accurate description of the data but we should still discuss what classification here means:

1. `1`: **BUY** the stock.
2. `0`: **NOT BUY** the stock.

## Data Wrangling

In [9]:
df1.head()

,Unnamed: 0,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,...,Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,2015 PRICE VAR [%],Class
0,PG,7.440100e+10,-0.0713,3.903000e+10,3.537100e+10,0.000000e+00,2.146100e+10,2.146100e+10,1.391000e+10,7.090000e+08,...,-0.0187,-0.0217,0.0359,0.0316,0.1228,0.0000,-0.1746,Consumer Defensive,-9.323276,0
1,VIPS,3.734148e+09,1.1737,2.805625e+09,9.285226e+08,1.083303e+08,3.441414e+08,7.939267e+08,1.345959e+08,1.214869e+07,...,NaN,NaN,NaN,NaN,NaN,1.6484,1.7313,Consumer Defensive,-25.512193,0
2,KR,9.837500e+10,0.0182,7.813800e+10,2.023700e+10,0.000000e+00,1.519600e+10,1.751200e+10,2.725000e+09,4.430000e+08,...,0.0618,0.0981,0.1886,0.3268,0.2738,0.0000,0.0234,Consumer Defensive,33.118297,1
3,RAD,2.552641e+10,0.0053,1.820268e+10,7.323734e+09,0.000000e+00,6.561162e+09,6.586482e+09,7.372520e+08,4.245910e+08,...,0.0211,-0.0510,-0.0189,0.1963,-0.0458,0.0000,-0.0060,Consumer Defensive,2.752291,1
4,GIS,1.790960e+10,0.0076,1.153980e+10,6.369800e+09,0.000000e+00,3.474300e+09,3.412400e+09,2.957400e+09,3.024000e+08,...,0.0257,0.0090,0.0215,0.0274,0.1025,0.0000,-0.0220,Consumer Defensive,12.897715,1


Our dataframe looks pretty good! Let's look for missing values!

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3808 entries, 0 to 3807
Columns: 225 entries, Unnamed: 0 to Class
dtypes: float64(222), int64(1), object(2)
memory usage: 6.5+ MB


Going to be very challenging in Pandas. Can't even generate a `ProfileReport` due to memory constraints. There is a simpler method. We can just review the information on Kaggle and add the Visualizations to this NoteBook. I left the step above for beginners.

In [11]:
df1.columns

Index(['Unnamed: 0', 'Revenue', 'Revenue Growth', 'Cost of Revenue',
       'Gross Profit', 'R&D Expenses', 'SG&A Expense', 'Operating Expenses',
       'Operating Income', 'Interest Expense',
       ...
       'Receivables growth', 'Inventory Growth', 'Asset Growth',
       'Book Value per Share Growth', 'Debt Growth', 'R&D Expense Growth',
       'SG&A Expenses Growth', 'Sector', '2015 PRICE VAR [%]', 'Class'],
      dtype='object', length=225)

This is great. Most of these Finance Variables appear to be numeric. To be certain we could slice the data and understand the decomposition.

In [12]:
dfsliced = []
for i in range(0,225,50):
    dfsliced.append(df1.iloc[:,i:i+49])

In [13]:
for i in dfsliced:
    print(i.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3808 entries, 0 to 3807
Data columns (total 49 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        3808 non-null   object 
 1   Revenue                           3764 non-null   float64
 2   Revenue Growth                    3572 non-null   float64
 3   Cost of Revenue                   3734 non-null   float64
 4   Gross Profit                      3756 non-null   float64
 5   R&D Expenses                      3672 non-null   float64
 6   SG&A Expense                      3749 non-null   float64
 7   Operating Expenses                3745 non-null   float64
 8   Operating Income                  3753 non-null   float64
 9   Interest Expense                  3745 non-null   float64
 10  Earnings before Tax               3728 non-null   float64
 11  Income Tax Expense                3742 non-null   float64
 12  Net In

When in doubt, compartmentalising is the way out. This method is better than Pandas Profiling for many good reasons. It's faster for one. We also have a goot idea of what the data looks like and if there are really any misgivings or such.

So far we appear to have 225 features of just numbers. A tonne of missing values and some leeway for Imputation. We could perhaps start with the missing values! Before moving on, if you scroll through the output you can probably tell there are 2 features `operatingCycle` and `CashConversionCycle`. These only have **1 value**. It would make more sense to drop these 2.

In [14]:
df1 = df1.drop(['operatingCycle','cashConversionCycle','Sector'],axis=1)
df2 = df2.drop(['operatingCycle','cashConversionCycle','Sector'],axis=1)
df3 = df3.drop(['operatingCycle','cashConversionCycle','Sector'],axis=1)
df4 = df4.drop(['operatingCycle','cashConversionCycle','Sector'],axis=1)
df5 = df5.drop(['operatingCycle','cashConversionCycle','Sector'],axis=1)

## How and What to Impute?

Imputation is a challenging and an age-old domain specific question. For this data there are a couple of important questions to ask so let's do some active brainstorming to develop a strategy.

What's this type of data? **These are features of stocks over 5 years**.

Great, so what's up with stocks? Let's take a small case scenario here.

Think about the Price to Sales Ratio for example. Imputing this with different stocks with `constants` obviously seems like a bad idea, should we impute with the `mean`?

Let's delve into this a bit more

In [15]:
df1['Price to Sales Ratio'].value_counts()

0.0000    163
0.0001      4
0.0002      3
0.2192      3
0.4628      2
         ... 
3.3554      1
0.4381      1
3.9070      1
7.5707      1
0.6875      1
Name: Price to Sales Ratio, Length: 3392, dtype: int64

In [16]:
df1['Price to Sales Ratio'].describe()

count     3654.000000
mean        44.972676
std        632.513736
min          0.000000
25%          0.807675
50%          1.879550
75%          4.195425
max      26397.558500
Name: Price to Sales Ratio, dtype: float64

Yikes. 26,397 for the max, 0 for the min and a standard deviation of 632. 

This is always a good step to look at. We seem to be dealing with our good old friend: **OUTLIERS**

Before we think about outlier removal let's talk about imputation. There are many good imputation strategies depending on your problem.

In this case we have MANY features that have their own nuances for imputation (not to mention the coupled issue of outliers). This is obviously more complex when we think about Finance here. Price to Sales Ratio is a good example. Let's say I'm the GameStop Stock (funny how this is related to current events), and I have a Price to Sales Ratio of 1.8, what does this really even mean? Well [this investopedia](https://www.investopedia.com/articles/fundamental/03/032603.asp#:~:text=The%20price%2Dto%2Dsales%20ratio%20(Price%2FSales%20or,the%20more%20attractive%20the%20investment.) link has more information for you but the short gist is $\frac{\text{Outstanding Shares*Sale Price}}{\text{Total Sales}}$ and intuitively, the lower this ratio, the more attractive your investment. Let's think a bit more about this, which stock has this high ratio?

In [17]:
df1['Price to Sales Ratio'].idxmax()

483

In [18]:
df1.iloc[483]

Unnamed: 0                 ACAD
Revenue                  120000
Revenue Growth          -0.8952
Cost of Revenue               0
Gross Profit             120000
                         ...   
Debt Growth                   0
R&D Expense Growth       1.2679
SG&A Expenses Growth     1.5745
2015 PRICE VAR [%]      14.4462
Class                         1
Name: 483, Length: 222, dtype: object

Well stock ACAD seems to have a **very high** Price to Sales Ratio. [ACAD](https://www.marketwatch.com/investing/stock/acad) short for Acadia Pharmaceuticals has a very high Price to Sales Ratio.

This is great, we see our outlier, there are real world situations which makes this likely (this is a pharmaceutical company).

Now we can think a bit more about imputation again. 

A good [imputation strategy](https://machinelearningmastery.com/knn-imputation-for-missing-values-in-machine-learning/) discusses kNN imputation. The best reason to use kNN imputation is due to the complexity of each feature. We can't impute by mean, median or constants. We can definitely however think about this as a Nearest Neighbours problem. 

That's a naive way of saying: If I have a stock XYZ that's kind of similar to stock ABC then impute the same value. 

Conceptually, this makes sense. Some stocks share a lot of similarities (in these features) and using them to fill out the blanks is smart.

In [19]:
#Drop Stock Name
df1 = df1.drop(df1.columns[0],axis=1)
df2 = df2.drop(df2.columns[0],axis=1)
df3 = df3.drop(df3.columns[0],axis=1)
df4 = df4.drop(df4.columns[0],axis=1)
df5 = df5.drop(df5.columns[0],axis=1)

In [20]:
imputer = KNNImputer(n_neighbors=5, weights='distance', metric='nan_euclidean', copy=True)

In [21]:
#Now we can make these train/test sets

#2014-2016 is our Train Set
#2017-2018 is our Test Set
train = pd.concat([df1, df2, df3])
test = pd.concat([df4, df5])

In [22]:
train

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,2015 PRICE VAR [%],Class,2016 PRICE VAR [%],2017 PRICE VAR [%]
0,7.440100e+10,-0.0713,3.903000e+10,3.537100e+10,0.000000e+00,2.146100e+10,2.146100e+10,1.391000e+10,7.090000e+08,1.449400e+10,...,-0.0217,0.0359,0.0316,0.1228,0.0000,-0.1746,-9.323276,0,NaN,NaN
1,3.734148e+09,1.1737,2.805625e+09,9.285226e+08,1.083303e+08,3.441414e+08,7.939267e+08,1.345959e+08,1.214869e+07,1.753823e+08,...,NaN,NaN,NaN,NaN,1.6484,1.7313,-25.512193,0,NaN,NaN
2,9.837500e+10,0.0182,7.813800e+10,2.023700e+10,0.000000e+00,1.519600e+10,1.751200e+10,2.725000e+09,4.430000e+08,2.270000e+09,...,0.0981,0.1886,0.3268,0.2738,0.0000,0.0234,33.118297,1,NaN,NaN
3,2.552641e+10,0.0053,1.820268e+10,7.323734e+09,0.000000e+00,6.561162e+09,6.586482e+09,7.372520e+08,4.245910e+08,2.502180e+08,...,-0.0510,-0.0189,0.1963,-0.0458,0.0000,-0.0060,2.752291,1,NaN,NaN
4,1.790960e+10,0.0076,1.153980e+10,6.369800e+09,0.000000e+00,3.474300e+09,3.412400e+09,2.957400e+09,3.024000e+08,2.707700e+09,...,0.0090,0.0215,0.0274,0.1025,0.0000,-0.0220,12.897715,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4792,6.099828e+07,0.0626,5.103888e+07,9.959402e+06,0.000000e+00,9.120526e+06,9.120526e+06,8.388760e+05,0.000000e+00,7.882260e+05,...,0.0000,0.0027,0.0441,0.0000,0.0000,0.0267,NaN,1,NaN,0.655807
4793,1.142630e+08,-0.0782,1.385500e+07,1.004080e+08,9.096000e+06,8.112600e+07,9.022200e+07,1.018600e+07,0.000000e+00,1.062300e+07,...,0.0000,-0.2194,-0.1115,-1.0000,-0.2553,-0.0964,NaN,0,NaN,-35.500002
4794,7.700000e+07,NaN,5.500000e+07,2.200000e+07,NaN,2.200000e+07,2.300000e+07,-1.000000e+06,1.000000e+06,-7.000000e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,14.840183
4795,1.646090e+08,-0.5692,1.372780e+08,2.733100e+07,0.000000e+00,1.871500e+07,1.871500e+07,8.616000e+06,-3.180000e+05,8.933000e+06,...,0.1894,0.2085,0.0012,0.0000,0.0000,0.0361,NaN,0,NaN,-2.854095


In [23]:
test

,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,Earnings before Tax,...,Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,2018 PRICE VAR [%],Class,2019 PRICE VAR [%]
0,6.505800e+10,-0.0037,3.263800e+10,3.242000e+10,0.000000e+00,1.865400e+10,1.865400e+10,1.376600e+10,4.650000e+08,1.838900e+10,...,0.0505,-0.0195,-0.0529,0.0012,0.0325,0.0000,-0.0156,4.975151,1,NaN
1,1.102060e+10,0.3525,8.557810e+09,2.462794e+09,2.733452e+08,8.201852e+08,2.056136e+09,4.066575e+08,1.245995e+07,3.893281e+08,...,0.9219,0.4764,0.5889,1.5821,0.3805,0.2141,0.1920,-56.320000,0,NaN
2,1.153370e+11,0.0501,8.950200e+10,2.583500e+10,0.000000e+00,1.916200e+10,2.238300e+10,3.452000e+09,5.220000e+08,2.932000e+09,...,-0.0490,0.0637,0.0769,0.0071,0.1654,0.0000,0.0678,-0.990449,0,NaN
3,2.292754e+10,0.1039,1.786283e+10,5.064707e+09,0.000000e+00,4.776995e+09,4.816124e+09,2.485830e+08,2.000650e+08,4.849100e+07,...,0.1063,-0.3365,0.0281,0.0502,-0.5295,0.0000,0.0427,-66.666666,0,NaN
4,1.561980e+10,-0.0570,1.005200e+10,5.567800e+09,0.000000e+00,2.888800e+09,3.069200e+09,2.498600e+09,2.951000e+08,2.312700e+09,...,0.0509,0.0494,0.0046,-0.0943,0.1246,0.0000,-0.0738,-31.280412,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4387,0.000000e+00,0.0000,0.000000e+00,0.000000e+00,0.000000e+00,3.755251e+06,3.755251e+06,-3.755251e+06,1.105849e+07,-1.482451e+07,...,0.0000,0.0000,-0.0508,-0.1409,-0.0152,0.0000,-0.2602,NaN,0,-90.962099
4388,5.560000e+05,-0.4110,0.000000e+00,5.560000e+05,4.759000e+06,5.071000e+06,9.830000e+06,-9.274000e+06,0.000000e+00,-9.170000e+06,...,0.3445,0.0000,-0.2323,-0.8602,0.0000,0.0352,-0.0993,NaN,0,-77.922077
4389,5.488438e+07,0.2210,3.659379e+07,1.829059e+07,1.652633e+06,7.020320e+06,8.672953e+06,9.617636e+06,1.239170e+06,8.416324e+06,...,0.1605,0.7706,0.2489,0.4074,-0.0968,0.2415,0.8987,NaN,0,-17.834400
4390,0.000000e+00,0.0000,0.000000e+00,0.000000e+00,1.031715e+07,4.521349e+06,1.664863e+07,-1.664863e+07,0.000000e+00,-1.664769e+07,...,0.8980,0.0000,0.1568,-0.2200,0.0000,2.7499,0.1457,NaN,0,-73.520000


In [27]:
train['Class'] = train['Class'].astype(object)
test['Class'] = test['Class'].astype(object)

In [29]:
train.shape

(12725, 223)

In [30]:
test.shape

(9352, 222)

In [35]:
train_cols = train.columns.values.tolist()
test_cols = test.columns.values.tolist()

In [36]:
list(set(train_cols) - set(test_cols))

['2015 PRICE VAR [%]', '2016 PRICE VAR [%]', '2017 PRICE VAR [%]']

In [37]:
train = train.drop(['2015 PRICE VAR [%]', '2016 PRICE VAR [%]', '2017 PRICE VAR [%]'], axis = 1)

In [38]:
train.shape

(12725, 220)

In [41]:
list(set(test_cols) - set(train_cols))

['2018 PRICE VAR [%]', '2019 PRICE VAR [%]']

In [42]:
test = test.drop(['2018 PRICE VAR [%]', '2019 PRICE VAR [%]'], axis = 1)

In [43]:
test.shape

(9352, 220)

In [46]:
train_cols = train.columns.values.tolist()
test_cols = test.columns.values.tolist()
list(set(test_cols) - set(train_cols))

[]

In [47]:
imputer.fit_transform(train);

array([[ 7.44010000e+10, -7.13000000e-02,  3.90300000e+10, ...,
         0.00000000e+00, -1.74600000e-01,  0.00000000e+00],
       [ 3.73414805e+09,  1.17370000e+00,  2.80562544e+09, ...,
         1.64840000e+00,  1.73130000e+00,  0.00000000e+00],
       [ 9.83750000e+10,  1.82000000e-02,  7.81380000e+10, ...,
         0.00000000e+00,  2.34000000e-02,  1.00000000e+00],
       ...,
       [ 7.70000000e+07,  2.10848594e-01,  5.50000000e+07, ...,
        -1.52313133e-03,  1.92092013e-01,  1.00000000e+00],
       [ 1.64609000e+08, -5.69200000e-01,  1.37278000e+08, ...,
         0.00000000e+00,  3.61000000e-02,  0.00000000e+00],
       [ 3.13270000e+07, -5.38000000e-02,  1.81650000e+07, ...,
         2.24000000e-02,  1.52200000e-01,  1.00000000e+00]])

In [48]:
imputer.transform(test);

array([[ 6.50580000e+10, -3.70000000e-03,  3.26380000e+10, ...,
         0.00000000e+00, -1.56000000e-02,  1.00000000e+00],
       [ 1.10206035e+10,  3.52500000e-01,  8.55781001e+09, ...,
         2.14100000e-01,  1.92000000e-01,  0.00000000e+00],
       [ 1.15337000e+11,  5.01000000e-02,  8.95020000e+10, ...,
         0.00000000e+00,  6.78000000e-02,  0.00000000e+00],
       ...,
       [ 5.48843810e+07,  2.21000000e-01,  3.65937920e+07, ...,
         2.41500000e-01,  8.98700000e-01,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.74990000e+00,  1.45700000e-01,  0.00000000e+00],
       [ 5.30190000e+07,  2.43000000e-02,  0.00000000e+00, ...,
         3.57700000e-01,  5.88000000e-01,  1.00000000e+00]])

At this point, we have a fully functional test and train set with some really conceptually useful features to predict stocks. Note that I fit the imputer on our training set and then just transform on the test set. This is really important as we do not want to use the test set to influence the test set itself (we need an unbiased measurement here). Now we can dive into the Machine Learning of this all!

# Machine Learning

## Outlier Detection

Remember how we looked at the data earlier in this Notebook? We saw some very 'extreme' values for just 1 feature. It would be hard to go over all of these features (though if you are a finance expert with free time, it would be super worth it). One can definitely speculate what's happening with our Inter-quartile range however. Typical Outliers don't survive above the 95-97% quantile and we should try to kill the 'large' ones and not the 'small' ones?

This is important to digest. Think back to our example of Price to Sales Ratio. Is it possible for this number to be 0? Well our analysis above showed 163 different stocks with a 0 P/S. Is this normal? 

And this analysis is trivial: We are only looking at one feature. It illustrates an important point for a thinking cap however. If we were to kill low quantiles we do squash useful information in our data (they may not be outliers). Can we however confirm this for the rest of the data? 

Going over it 1 by 1 will be taxing. Instead we can choose to make a decision here as data scientists. We kill both quantiles. Does this hurt our predictions? **Not likely**. We have a lot of observations and 220 features (more on this later). It does takeaway from the 'perfection' of our data.

In [51]:
upper_quantile_train = train.quantile(0.97)
upper_quantile_test = test.quantile(0.97)
lower_quantile_train = train.quantile(0.03)
lower_quantile_test = test.quantile(0.03)

top_outliers_train = (train > upper_quantile_train)
top_outliers_test = (test > upper_quantile_test)
bottom_outliers_train = (train < lower_quantile_train)
bottom_outliers_test = (test < lower_quantile_test)

train = train.mask(top_outliers_train,upper_quantile_train, axis=1)
test = test.mask(top_outliers_test,upper_quantile_test,axis=1)
train = train.mask(bottom_outliers_train,lower_quantile_train,axis=1)
test = test.mask(bottom_outliers_test,lower_quantile_test,axis=1)

## Predictions

Let's run some algorithms